# Decision Tree

In [3]:
training_data = [['Green',3,'Mango'],['Yellow',3,'Mango'],['Red',1,'Grape'],['Red',1,'Grape'],['Yellow',3,'Lemon']]
header = ['Color','Diameter','Label']


In [4]:
def unique_values(rows , col):
    return set([row[col] for row in rows])

In [5]:
def class_count(rows):
    counts={}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label]=0
        counts[label]+=1
    return counts

In [6]:
def isnumeric(value):
    return isinstance(value,int) or isinstance(value,float)

In [29]:
class Question:
    
    def __init__(self, col, value):
        self.column = col
        self.value = value
    
    def match(self, example):
        val = example[self.column]
        if isnumeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        condition = "=="
        if(isnumeric(self.value)):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))

In [30]:
def partition(rows, question):
    true_rows , false_rows = [],[]
    for row in rows:
        if(question.match(row)):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows
    

In [31]:
def gini(rows):
    counts = class_count(rows)
    impurity = 1
    for lbl in counts:
        prob_lbl = counts[lbl]/ float(len(rows))
        impurity -= prob_lbl**2
    return impurity

In [32]:
def info_gain(left,right,current_uncertainty):
    p = float(len(left))/ (len(left) + len(right))
    return current_uncertainty - p*gini(left) - (1-p)*gini(right)

In [33]:
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1
    
    for col in range(n_features):
        values = set([row[col] for row in rows])
        
        for val in values:
            question = Question(col,val)
            true_rows, false_rows = partition(rows,question)
            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            if gain >= best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question

    

In [34]:
class Leaf:
    def __init__(self,rows):
        self.predictions = class_count(rows)

In [46]:
class Decision_node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [47]:
def build_tree(rows):
    gain , question  =  find_best_split(rows)
    if gain==0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch =  build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    return Decision_node(question, true_branch, false_branch)

In [48]:
def print_tree(node, spacing=""):
    if isinstance(node,Leaf):
        print(spacing+"Predict",node.predictions)
        return 
    print(spacing+str(node.question))
    print(spacing+'----> True:')
    print_tree(node.true_branch, spacing + " ")
    print(spacing+'----> False:')
    print_tree(node.false_branch, spacing + " ")
    

In [49]:
def classify(row,node):
    if isinstance(node,Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else :
        return classify(row, node.false_branch)
    

In [53]:
def print_leaf(counts):
    total = sum(counts.values())*1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [55]:
if __name__ =='__main__':
    my_tree = build_tree(training_data)
    print_tree(my_tree)
    testing_data = [['Green',3,'Mango'],['Yellow',4,'Mango'],['Red',2,'Grape'],['Red',1,'Grape'],['Yellow',3,'Lemon']]
    for row in testing_data:
        print("Actual: %s. Predicted: %s" % (row[-1],print_leaf(classify(row,my_tree))))
        

Is Diameter >= 3?
----> True:
 Is Color == Yellow?
 ----> True:
  Predict {'Mango': 1, 'Lemon': 1}
 ----> False:
  Predict {'Mango': 1}
----> False:
 Predict {'Grape': 2}
Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Mango. Predicted: {'Mango': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}
